In [1]:
import numpy as np
import scipy.integrate
import plotly.graph_objects
from ipywidgets import interact, IntSlider, Checkbox


In [2]:
mydefault = plotly.graph_objects.layout.Template()
mydefault.layout.hovermode = False
mydefault.layout.scene.hovermode = False
mydefault.layout.xaxis.showspikes = False
mydefault.layout.yaxis.showspikes = False
mydefault.layout.xaxis.showgrid = False
mydefault.layout.yaxis.showgrid = False
mydefault.layout.xaxis.showline = True
mydefault.layout.yaxis.showline = True
mydefault.layout.scene.xaxis.showspikes = False
mydefault.layout.scene.yaxis.showspikes = False
mydefault.layout.scene.zaxis.showspikes = False
mydefault.layout.dragmode = "pan"
plotly.io.templates["mydefault"] = mydefault
plotly.io.templates.default = "mydefault"
colors = plotly.colors.qualitative.D3
# To see the other choices for Plotly's built-in lists of colors: 
#plotly.colors.qualitative.swatches()

The finished HPG model: $\displaystyle \qquad \begin{cases}
    H' = \frac{1}{1 + G^n} - 0.2H \\
    P' = H - 0.2P \\
    G' = P - 0.2G
\end{cases} $


In [3]:
figure = plotly.graph_objects.FigureWidget()
figure.layout.title = r"Production of GnRH is a sigmoid function of H"
figure.add_scatter(mode="lines")
gnrh_production_plot = figure.data[-1]
figure.add_annotation(x=2, y=1, font_size=20, font_color="black", xanchor="center", yanchor="middle", showarrow=False, ax=0, ay=0)
gnrh_production_text = figure.layout.annotations[-1]
figure.layout.showlegend = False
figure.layout.xaxis.title.text = r"G (estrogen level)"
figure.layout.yaxis.title.text = r"Production rate of GnRH (H)"
figure.layout.xaxis.range = [0, 3]
figure.layout.yaxis.range = [0, 1.3]
figure.layout.xaxis.rangemode = "tozero"
figure.layout.yaxis.rangemode = "tozero"

@interact(n=IntSlider(min=1, max=20, value=2))
def update(n):
    G = np.linspace(0, 3, 100)
    sigmoid = 1/(1 + G**n)
    with figure.batch_update():
        gnrh_production_plot.update(x=G, y=sigmoid)
        gnrh_production_text.text = fr"$f(G) = \frac{{1}}{{1 + G^{n}}}$"

figure

interactive(children=(IntSlider(value=2, description='n', max=20, min=1), Output()), _dom_classes=('widget-int…

FigureWidget({
    'data': [{'mode': 'lines',
              'type': 'scatter',
              'uid': '312c1926-…

**Time series of the HPG model:** 

In [4]:
figure = plotly.graph_objects.FigureWidget()
figure.layout.title = r"Time series plots of the HPG model"
figure.add_scatter(mode="lines", name="GnRH (H)")
H_plot = figure.data[-1]
figure.add_scatter(mode="lines", name="FSH/LH (P)")
P_plot = figure.data[-1]
figure.add_scatter(mode="lines", name="Estrogen/progesterone (G)")
G_plot = figure.data[-1]
#figure.layout.showlegend = False
figure.layout.xaxis.title.text = r"t (time)"
figure.layout.yaxis.title.text = r"Hormone levels"
figure.layout.xaxis.range = [0, 200]
figure.layout.yaxis.range = [0, 5]
figure.layout.xaxis.rangemode = "tozero"
figure.layout.yaxis.rangemode = "tozero"

@interact(n=IntSlider(min=1, max=20, value=2))
def update(n):
    def vectorfield(t, state):
        H, P, G = state
        return ( 1/(1 + G**n) - 0.2*H, H - 0.2*P, P - 0.2*G )
    t_range = (0, 200)
    initial_state = (0.2, 0.5, 2)
    solution = scipy.integrate.solve_ivp(vectorfield, t_range, initial_state, 
            vectorized=True, atol=1e-9, rtol=1e-12)
    H, P, G = solution.y
    with figure.batch_update():
        H_plot.update(x=solution.t, y=H)
        P_plot.update(x=solution.t, y=P)
        G_plot.update(x=solution.t, y=G)

figure

interactive(children=(IntSlider(value=2, description='n', max=20, min=1), Output()), _dom_classes=('widget-int…

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'GnRH (H)',
              'type': 'scatter…

**A trajectory of the HPG model:** 

In [5]:
def hpg_trajectory():
    figure = plotly.graph_objects.FigureWidget()
    figure.layout.showlegend = True
    figure.layout.scene.xaxis.title.text = "H (GnRH)"
    figure.layout.scene.yaxis.title.text = "P (FSH and LH)"
    figure.layout.scene.zaxis.title.text = "G (estrogen)"
    figure.add_scatter3d(mode="lines", name="Trajectory")
    trajectory = figure.data[-1]
    figure.add_scatter3d(mode="lines", line_width=5, name="Attractor")
    attractor = figure.data[-1]

    @interact(n=IntSlider(min=1, max=20, value=12), 
              show_lca=Checkbox(value=False, description="Show attractor"))
    def update(n, show_lca):
        def vectorfield(t, state):
            H, P, G = state
            return ( 1/(1 + G**n) - 0.2*H, H - 0.2*P, P - 0.2*G )
        t_range = (0, 800)
        initial_state = (0.2, 0.5, 2)
        solution = scipy.integrate.solve_ivp(vectorfield, t_range, initial_state, 
                vectorized=True, atol=1e-9, rtol=1e-12)
        H, P, G = solution.y
        with figure.batch_update():
            end = int(H.shape[0] * (0.8 if show_lca else 1))
            trajectory.x = H[:end]
            trajectory.y = P[:end]
            trajectory.z = G[:end]
            attractor.x = H[end:]
            attractor.y = P[end:]
            attractor.z = G[end:]
            attractor.visible = show_lca
    return figure
hpg_trajectory()


interactive(children=(IntSlider(value=12, description='n', max=20, min=1), Checkbox(value=False, description='…

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Trajectory',
              'type': 'scatt…